In [2]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from typing import Callable

# Funkcje odległości

def euclidean_distance(a: list, b: list) -> float:
    """Oblicza odległość euklidesową między dwoma punktami a i b."""
    a, b = np.array(a), np.array(b)
    diff = np.subtract(a, b)
    diff_squared = np.square(diff)
    return np.sqrt(diff_squared.sum())

def manhattan_distance(a: list, b: list) -> float:
    """Oblicza odległość Manhattan między dwoma punktami a i b."""
    a, b = np.array(a), np.array(b)
    diff = np.subtract(a, b)
    absolute_diff = np.abs(diff)
    return absolute_diff.sum()

def cosine_similarity(a: list, b: list) -> float:
    """Oblicza podobieństwo kosinusowe między dwoma wektorami a i b."""
    a, b = np.array(a), np.array(b)
    dot_product = np.dot(a, b)
    norm_a = np.sqrt(np.square(a).sum())
    norm_b = np.sqrt(np.square(b).sum())
    return 1 - dot_product / (norm_a * norm_b)

# Funkcje wag

def constant_weight(_) -> float:
    """Funkcja wagowa, zwraca stałą wagę 1."""
    return 1.0

def linear_weight(distance: float) -> float:
    """Funkcja wagowa, zwraca odległość jako wagę."""
    return distance

def inverse_weight(distance: float) -> float:
    """Funkcja wagowa, zwraca odwrotność odległości jako wagę."""
    return 1 / distance if distance != 0 else np.inf

# Klasyfikator KNN

def knn_classifier(dataset: np.ndarray, sample: list, k: int, distance_func: Callable[[list, list], float],
                   weight_func: Callable[[float], float], num_classes: int):
    """Implementacja klasyfikatora k najbliższych sąsiadów (KNN)."""
    class_scores = np.zeros(num_classes, dtype=float)

    # Obliczenie odległości i przypisanie klas dla wszystkich obiektów w zbiorze danych
    distances_and_classes = []
    for data_point, data_class in dataset:
        distance = distance_func(data_point, sample)
        distances_and_classes.append((distance, data_class))

    # Posortowanie obiektów według odległości i wybranie k najbliższych
    distances_and_classes.sort(key=lambda x: x[0])
    k_nearest_neighbors = distances_and_classes[:k]

    # Sumowanie ważonych odległości dla każdej klasy
    for distance, obj_class in k_nearest_neighbors:
        class_scores[int(obj_class)] += weight_func(distance)

    # Zwrócenie klasy z najwyższym wynikiem lub listy wyników dla wszystkich klas
    return np.argmax(class_scores)

# Wczytanie danych

def load_dataset(seed: int = 0) -> tuple:
    """Wczytuje zbiór danych i dzieli go na zbiór treningowy i testowy."""
    dataset = load_iris()
    feature_vals_train, feature_vals_test, class_train, class_test = train_test_split(
        dataset.data, dataset.target, test_size=0.3, random_state=seed)

    train_data = pd.DataFrame()
    test_data = pd.DataFrame()
    train_data['features'], train_data['class'] = [*feature_vals_train], [*class_train]
    test_data['features'], test_data['class'] = [*feature_vals_test], [*class_test]

    return train_data.to_numpy(), test_data.to_numpy()

# Przygotowanie danych treningowych i testowych

train_data, test_data = load_dataset(seed=1)

# Testowanie klasyfikatora

accuracy = 0.0
for sample, sample_class in test_data:
    prediction = knn_classifier(dataset=train_data, sample=sample, k=3, distance_func=cosine_similarity,
                                weight_func=inverse_weight, num_classes=3)
    if prediction == sample_class:
        accuracy += 1

accuracy /= len(test_data)
print('Accuracy: ', accuracy)

Accuracy:  0.9777777777777777


Kod, który przedstawiam, jest implementacją algorytmu k najbliższych sąsiadów (KNN) w języku Python. Algorytm ten jest popularnym modelem klasyfikacji, który przewiduje przynależność obiektu do jednej z klas na podstawie podobieństwa do jego najbliższych sąsiadów.

Pierwszą częścią kodu jest wczytanie danych. Wykorzystujemy zbiór danych "iris" z pakietu scikit-learn. Następnie dzielimy go na zbiór treningowy i testowy, z których będziemy korzystać do trenowania i oceny klasyfikatora.

Następnie definiujemy trzy funkcje odległości: odległość euklidesową, odległość Manhattan oraz podobieństwo kosinusowe. Funkcje te służą do porównywania odległości lub podobieństwa między dwoma punktami w przestrzeni cech. Wybieramy jedną z tych funkcji jako metrykę odległości w algorytmie KNN.

Kolejnym krokiem jest zdefiniowanie trzech funkcji wagowych: stałej wagi, wagi liniowej i wagi odwrotnej. Te funkcje określają, jak bardzo odległość do sąsiadów ma wpływ na wynik klasyfikacji. Można dostosować funkcję wagową w zależności od potrzeb i charakterystyki danych.

Następnie mamy główną funkcję knn_classifier, która implementuje algorytm KNN. Przyjmuje ona zbiór danych treningowych, obiekt do klasyfikacji, parametr k określający liczbę najbliższych sąsiadów, funkcję odległości, funkcję wagową oraz liczbę klas w zbiorze danych.

W tej funkcji najpierw inicjalizujemy tablicę class_scores, która przechowuje sumy wyników poszczególnych klas. Następnie obliczamy odległość lub podobieństwo między obiektem testowym a każdym obiektem treningowym. Wyniki te przechowujemy w liście distances_and_classes, która zawiera odległość i przypisaną klasę dla każdego obiektu treningowego.

Po obliczeniu odległości sortujemy obiekty według odległości i wybieramy k najbliższych sąsiadów. Następnie sumujemy ważone odległości dla każdej klasy na podstawie funkcji wagowej. Ostatecznie zwracamy indeks klasy z największą sumą jako wynik klasyfikacji.

W dalszej części kodu inicjalizujemy dane treningowe i testowe za pomocą funkcji load_dataset. Następnie przeprowadzamy testowanie klasyfikatora na danych testowych. Dla każdego obiektu testowego wywołujemy funkcję knn_classifier i porównujemy wynik z rzeczywistą klasą. Obliczamy dokładność klasyfikacji przez podzielenie liczby poprawnych predykcji przez liczbę obiektów testowych.

Na koniec wypisujemy dokładność klasyfikacji na konsoli.